<a href="https://colab.research.google.com/github/Bkoome/KAPS-PROJECT/blob/main/U_Net_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

In [2]:
image_dir = '/content/drive/MyDrive/potato_project/Training'
mask_dir = '/content/drive/MyDrive/potato_project/Training'

In [ ]:

def load_and_preprocess_data(image_dir, mask_dir, target_size=(256, 256)):
    image_files = os.listdir(image_dir)
    images = []
    masks = []

    for file in image_files:
        if file.endswith('.jpg'):  # Assuming images are in JPG format
            # Load and preprocess image
            image_path = os.path.join(image_dir, file)
            image = load_img(image_path, target_size=target_size)
            image = img_to_array(image) / 255.0  # Normalize pixel values
            images.append(image)

            # Load and preprocess mask (assuming masks are in a separate directory)
            mask_path = os.path.join(mask_dir, file.replace('.jpg', '_mask.jpg'))
            mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
            mask = img_to_array(mask) / 255.0
            masks.append(mask)

    return np.array(images), np.array(masks)

# Load and preprocess data
X, y = load_and_preprocess_data(image_dir, mask_dir)

In [ ]:
# Encode mask labels (assuming the masks are grayscale)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.flatten())
y_categorical = to_categorical(y_encoded, num_classes=3)  # Update num_classes to 3


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:

# Define the U-Net model
def unet_model(input_shape=(256, 256, 3), num_classes=3):
    inputs = layers.Input(shape=input_shape)

    # Contracting Path
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Expanding Path
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    up1 = layers.UpSampling2D(size=(2, 2))(conv2)
    up1 = layers.concatenate([conv1, up1], axis=-1)

    # Output layer
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(up1)

    model = models.Model(inputs=inputs, outputs=outputs, name='unet_model')
    return model

# Instantiate the model
model = unet_model()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
batch_size = 32
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=10,
                    validation_data=datagen.flow(X_val, y_val, batch_size=batch_size),
                    validation_steps=len(X_val) // batch_size)


In [ ]:
# Assuming `image_to_predict` is the new orthomosaic image you want to predict
predicted_masks = model.predict(np.expand_dims(image_to_predict, axis=0))

In [ ]:
# Assuming `color_mapping` is a dictionary mapping class indices to colors
color_mapping = {
    0: [255, 0, 0],  # Red for Early Blight
    1: [0, 255, 0],  # Green for Late Blight
    2: [0, 0, 255]   # Blue for Healthy Crops
}

In [ ]:
# Overlay colors on the original image based on predicted masks
segmented_image = np.zeros_like(image_to_predict)

for class_index, color in color_mapping.items():
    class_mask = predicted_masks[..., class_index]
    segmented_image[class_mask > 0.5] = color

# Display or save the segmented image